In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Github/

In [ ]:
!git init

In [ ]:
!git pull https://github.com/rohit-khoiwal-30/msgmt.git

remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 2), reused 2 (delta 1), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/rohit-khoiwal-30/msgmt
 * branch            HEAD       -> FETCH_HEAD
Updating 4bf662f..a5224bf
Checking out files: 100% (4/4), done.
Fast-forward
 bertTripleLoss.ipynb  | 366 ++++++++++++++++++++++++++++++++++++++
 sentence-t5EUCL.ipynb |  33 ----
 useCONCT -.ipynb      | 366 ++++++++++++++++++++++++++++++++++++++
 useTripleLoss .ipynb  | 472 ++++++++++++++++++++++++++++++++++++++++++++++++++
 4 files changed, 1204 insertions(+), 33 deletions(-)
 create mode 100644 bertTripleLoss.ipynb
 delete mode 100644 sentence-t5EUCL.ipynb
 create mode 100644 useCONCT -.ipynb
 create mode 100644 useTripleLoss .ipynb


In [5]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("evaluation.csv")

In [6]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from keras.regularizers import l2

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install -q tensorflow_hub
    import tensorflow_hub as hub
    
try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install -q keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np


# %pip install -q transformers
# %pip install -q -U tensorflow-text
# %pip install -q tf-models-official==2.7.0

import math as m

In [7]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [8]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, y_hat = [], [], []
    all_groups = list(range(in_groups[0].shape[0]))
    for match_group in [True, False]:
        group_idx = np.random.choice(all_groups, size = batch_halfsize)
        text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
        if match_group:
            b_group_idx = group_idx
            y_hat += [1]*batch_halfsize
        else:
            # anything but the same group
            non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
            b_group_idx = non_group_idx
            y_hat += [0]*batch_halfsize
            
        reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, y_hatstack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack], y_hatstack

def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    return loss

In [9]:
def accuracy(y_true, y_pred):
    results = (y_pred <= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [10]:
def get_model():
    shared_embedding_layer1 = hub.KerasLayer(fine_tuned_module_object, trainable=True, name="first")
    shared_embedding_layer2 = hub.KerasLayer(fine_tuned_module_object, trainable=True, name="second")

    left_input = tf.keras.Input(shape=(), dtype=tf.string)
    right_input = tf.keras.Input(shape=(), dtype=tf.string)

    embedding_left_output= shared_embedding_layer1(left_input)
    a = tfl.Dropout(0.3)(embedding_left_output)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dropout(rate=0.2)(a)
    a = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    # a = tfl.BatchNormalization()(a)
    # a = tfl.Activation('relu')(a)
    a = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(a)

    embedding_right_output= shared_embedding_layer2(right_input)
    b = tfl.Dropout(0.3)(embedding_right_output)
    b = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(b)
    # b = tfl.BatchNormalization()(b)
    # b = tfl.Activation('relu')(b)
    b = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(b)

    distance = tfl.Lambda(euclidean_distance)([a, b])
    model = Model([left_input, right_input], distance)
    return model

In [11]:
model = get_model()

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 first (KerasLayer)             (None, 512)          256797824   ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 512)          0           ['first[0][0]']                  
                                                                                                  
 dense (Dense)                  (None, 64)           32832       ['dropout[0][0]']                
                                                                                              

In [13]:
model.compile(optimizer="adam", loss = contrastive_loss, metrics = [tf.metrics.BinaryAccuracy()])

In [ ]:
loss_history = model.fit(siam_gen([df["text"], df["reason"]], 64), 
                         validation_data = siam_gen([df_test["text"], df_test["reason"]], 64),
                         steps_per_epoch=50, validation_steps=10, epochs = 50, 
                         verbose = True, use_multiprocessing=True)

Epoch 1/50
50/50 [==============================] - 22s 211ms/step - loss: 0.5721 - binary_accuracy: 0.4934 - val_loss: 0.4786 - val_binary_accuracy: 0.5000
Epoch 2/50
50/50 [==============================] - 9s 186ms/step - loss: 0.4385 - binary_accuracy: 0.4469 - val_loss: 0.4294 - val_binary_accuracy: 0.4859
Epoch 3/50
50/50 [==============================] - 9s 186ms/step - loss: 0.3811 - binary_accuracy: 0.3147 - val_loss: 0.3519 - val_binary_accuracy: 0.2734
Epoch 4/50
50/50 [==============================] - 9s 188ms/step - loss: 0.3150 - binary_accuracy: 0.1937 - val_loss: 0.3126 - val_binary_accuracy: 0.2313
Epoch 5/50
50/50 [==============================] - 9s 187ms/step - loss: 0.2682 - binary_accuracy: 0.1331 - val_loss: 0.2973 - val_binary_accuracy: 0.2484
Epoch 6/50
22/50 [============>.................] - ETA: 4s - loss: 0.2473 - binary_accuracy: 0.1257

In [ ]:
y_preds = model.predict([df["text"], df["reason"]])
accuracy(df["label"],y_preds)

In [ ]:
y_preds = model.predict([df_test["text"], df_test["reason"]])
accuracy(df_test["label"],y_preds)

In [ ]:
model.save("models/useEUCL/")